In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu12==24.4.*
!pip install bertopic==0.16.2
#!pip install bertopic==0.16.0
from bertopic import BERTopic
from bertopic.dimensionality import BaseDimensionalityReduction
import cuml
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import randint
from sklearn.feature_extraction.text import CountVectorizer
#import seaborn as sns

colors = {
    1: ['#003f5c'],
    2: ['#003f5c', '#ffa600'],
    3: ['#003f5c', '#bc5090', '#ffa600'],
    4: ['#003f5c', '#7a5195', '#ef5675', '#ffa600'],
    5: ['#003f5c', '#58508d', '#bc5090', '#ff6361', '#ffa600'],
    6: ['#003f5c', '#444e86', '#955196', '#dd5182','#ff6e54', '#ffa600'],
    7: ['#003f5c', '#374c80', '#7a5195', '#bc5090', '#ef5675', '#ff764a', '#ffa600'],
    8: ['#003f5c', '#2f4b7c', '#665191', '#a05195', '#d45087', '#f95d6a', '#ff7c43', '#ffa600']
    }

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.6/126.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.9/810.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existi

In [ ]:
directory = f"/content/drive/MyDrive/Nikta"
path = os.path.join(directory, 'Data', '50k_random_seed_42_embeddings.npy')
embeddings = np.load(path)

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stopwords = stopwords.words('english')

def preprocess_text(
        text, stop_words = stopwords, lowercase = True, remove_usernames = True, remove_URLs = True, remove_domains_path = True,
        remove_punctuations = True, remove_digits = True, reduce_consecutive_characters = True,
        reduce_immediate_duplicate_words = True, spell_check = True, remove_stop_words = True):

    # Convert text to lowercase
    if(lowercase):
        text = text.lower()

    # Remove @usernames
    if(remove_usernames):
        text = re.sub(r'@\w+', '', text)

    # URL removal
    if(remove_URLs):
        text = re.sub(r'\S*http\S+', '', text)
        text = re.sub(r'\S*www.\S+', '', text)
        text = re.sub(r'\S*/\S+', '', text)

    # Domain/Path Removal
    if(remove_domains_path):
        text = re.sub(r'\S+\.\S+' , '', text)
        text = re.sub(r'\S*\\\S+' , '', text)

    text = re.sub(r'\S*bot\S*', '', text)

    # Consecutive Character Reduction
    if(reduce_consecutive_characters):
        text = re.sub(r'(.)\1{2,}', r'\1', text)

    # Immediate Duplicate Word Removal
    if(reduce_immediate_duplicate_words):
        text = re.sub(r'\b(\w+)(\s+\1){1,}\b', r'\1', text)

    # Spell Checker
    if(spell_check):
        spell = SpellChecker()
        corrected_text = []
        for word in text.split():
            corrected_word = spell.correction(word)
            corrected_text.append(corrected_word if corrected_word else word)
        text = ' '.join(corrected_text)


    # Remove punctuations
    if(remove_punctuations):
        text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove digits
    if(remove_digits):
        text = re.sub(r'\d+', '', text)

    # Remove stop words
    if(remove_stop_words and stop_words):
        text = ' '.join([word for word in text.split() if word not in stop_words])

    # Whitespace Normalization
    text = re.sub(r'\s+', ' ', text).strip()

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
directory = f"/content/drive/MyDrive/Nikta/Data"
path = os.path.join(directory, 'deduplicated_data2.csv')

posts = pd.read_csv(path, encoding='utf-8')
posts = posts.sample(n=50000, random_state=42)
print(f"first random index: {posts.index.values[0]}")

Titles = posts['Title'].to_list()
Bodies = posts['Body'].to_list()


Titles = [preprocess_text(
        message, stop_words = None, lowercase = False, remove_usernames = True, remove_URLs = True, remove_domains_path = True,
        remove_punctuations = False, remove_digits = True, reduce_consecutive_characters = True,
        reduce_immediate_duplicate_words = True, spell_check = False, remove_stop_words = False) for message in Titles]

Bodies = [preprocess_text(
        message, stop_words = None, lowercase = False, remove_usernames = True, remove_URLs = True, remove_domains_path = True,
        remove_punctuations = False, remove_digits = True, reduce_consecutive_characters = True,
        reduce_immediate_duplicate_words = True, spell_check = False, remove_stop_words = False) for message in Bodies]


#Titles = [' '.join(title.split()[:40]) for title in posts['Title'].tolist()]
#Bodies = [' '.join(body.split()[:60]) for body in Bodies]


messages = [f"Title: {Titles[i]}, Body: {Bodies[i]}" for i in range(posts.shape[0])]

first random index: 194968


In [ ]:
#messages = posts['CombinedText'].to_list()
cleaned_messages = [preprocess_text(
        message, stop_words = stopwords, lowercase = True, remove_usernames = True, remove_URLs = True, remove_domains_path = True,
        remove_punctuations = True, remove_digits = True, reduce_consecutive_characters = False,
        reduce_immediate_duplicate_words = True, spell_check = False, remove_stop_words = True) for message in messages]

In [ ]:
# This needs cuml
from sklearn.feature_extraction.text import CountVectorizer

empty_dimensionality_model = BaseDimensionalityReduction()
#kmeans = cuml.KMeans(n_clusters=40, random_state=0)
vectorizer_model = CountVectorizer(ngram_range=(1, 2), max_df = 0.4)
k = 40

kmeans = cuml.KMeans(n_clusters=k, random_state=0) # this needs graphics card and cuda
#kmeans = KMeans(n_clusters=k, random_state=0) # Slow CPU version
topic_model = BERTopic(umap_model=empty_dimensionality_model, hdbscan_model=kmeans, verbose=True, top_n_words=10, vectorizer_model = vectorizer_model)
topics, _ = topic_model.fit_transform(cleaned_messages, embeddings)
topic_model.get_topic_info()

2024-06-11 20:59:09,923 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-11 20:59:09,924 - BERTopic - Dimensionality - Completed ✓
2024-06-11 20:59:11,601 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-11 20:59:19,295 - BERTopic - Cluster - Completed ✓
2024-06-11 20:59:19,314 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-11 20:59:38,138 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,0,2995,0_activerecordbase hasandbelongstomany_habtm_t...,"[activerecordbase hasandbelongstomany, habtm, ...",[title set rails models hasandbelongstomany re...
1,1,2947,1_gt running_service host_servicename gt_gt se...,"[gt running, service host, servicename gt, gt ...",[title collecting array records db using rails...
2,2,2774,2_instance methods_def puts_singleton_class va...,"[instance methods, def puts, singleton, class ...",[title preferred rubyist way declaring access ...
3,3,2249,3_contentfor_selecttag_precode render_ltsection,"[contentfor, selecttag, precode render, ltsect...",[title rails array html table body pi array co...
4,4,2232,4_render actionpack_rendertemplate_actionpack ...,"[render actionpack, rendertemplate, actionpack...",[title hartls rails tutorial rails section par...
5,5,2192,5_tr tr_left join_rows width_rows loops,"[tr tr, left join, rows width, rows loops, wid...",[title database structure calculated rows body...
6,6,2078,6_title rspec_post create_fillin_cgt,"[title rspec, post create, fillin, cgt, agt, b...",[title post create test user count failing cha...
7,7,1836,7_stylecolorslategraygt ampgt_stylecolorslateg...,"[stylecolorslategraygt ampgt, stylecolorslateg...",[title converting ruby array array hash body p...
8,8,1810,8_precodevalidates_unlessgt_custom validation_...,"[precodevalidates, unlessgt, custom validation...",[title rails nested strong parameters throwing...
9,9,1745,9_yes checking_checking yes_rbenv_install ruby,"[yes checking, checking yes, rbenv, install ru...",[title get failed build gem native im trying i...


In [ ]:
info = topic_model.get_topic_info()
info.columns

Index(['Topic', 'Count', 'Name', 'Representation', 'Representative_Docs'], dtype='object')

In [ ]:
reps = info['Representation'].value_counts()
(reps.index.values)

array([list(['activerecordbase hasandbelongstomany', 'habtm', 'tr tr', 'manytomany', 'hasmany association', 'associations body', 'many many', 'polymorphic association', 'title activerecord', 'gtemailgt']),
       list(['gt running', 'service host', 'servicename gt', 'gt servicename', 'statusmessage gt', 'statusmessage', 'servicename', 'running statusmessage', 'fieldsfor', 'strong parameters']),
       list(['instance methods', 'def puts', 'singleton', 'class variable', 'method ruby', 'method end', 'precodeclass foo', 'class methods', 'class instance', 'myclass']),
       list(['contentfor', 'selecttag', 'precode render', 'ltsection', 'search form', 'bmargindiv colmd', 'ampquot', 'classinputgroup', 'bmargindiv', 'classinputgroup bmargindiv']),
       list(['render actionpack', 'rendertemplate', 'actionpack render', 'xrails', 'title nomethoderror', 'arbre', 'fillin', 'call gem', 'trigger allalter', 'allalter table']),
       list(['tr tr', 'left join', 'rows width', 'rows loops', 'width 

In [ ]:
reps.value_counts()

count
1    40
Name: count, dtype: int64

In [ ]:
posts['topic'] = topic_model.topics_

In [ ]:
topic_names = [
    "Rails Associations Complexity",
    "Service Monitoring & Status",
    "Ruby OOP Foundations",
    "Form Rendering Techniques",
    "Error Handling in Rails",
    "Database Query Optimization",
    "Testing with RSpec",
    "Ruby Collections and Data Structures",
    "Rails Validations and Errors",
    "Ruby Environment and Build Issues",
    "UI Enhancements with Bootstrap",
    "XML and HTML Parsing with Nokogiri",
    "Dynamic Forms and Selections",
    "AJAX in Rails",
    "Dependency and Build Issues in Ruby",
    "Compiler and Runtime Warnings",
    "API Integration with SOAP and Twilio",
    "Data Structures and Algorithms",
    "Database Migrations in Rails",
    "User Authentication with Devise",
    "Network and IP Address Handling",
    "Server Configuration with Vagrant and Docker",
    "Media Handling in Rails",
    "Network Security and Debugging",
    "Module Compatibility and Webpack",
    "Routing in Rails",
    "JavaScript and jQuery Best Practices",
    "System Monitoring and Performance",
    "Automated Testing with Capybara and Selenium",
    "Remote Management and Deployment",
    "Social Media Integration",
    "Deployment Automation with Capistrano",
    "Serialization in Rails",
    "Email Integration in Rails",
    "CSS and Web Design",
    "Regular Expressions in Ruby",
    "Search Optimization with Elasticsearch",
    "Payment Processing with Stripe and PayPal",
    "Version Control with Git",
    "Mapping and Geolocation"
]

In [ ]:
posts['topic_name'] = posts['topic'].apply(lambda x: topic_names[x])

In [ ]:
i = randint(0, posts.shape[0])
print(f"Topic: {posts.iloc[i]['topic_name']}")
print('Title:', (posts.iloc[i]['Title']))
print('Body:', (posts.iloc[i]['Body']))


Topic: Ruby OOP Foundations
Title: Ruby - Using nil parameters to support different types of an class
Body: <p>In Ruby, say I have a class called <code>Song</code>. I would like to know how correcty to write the <code>initialize</code> so that it supports 2 different types of. For example:</p>

<p><code>song_full = Song.new(fromSomeCloudStorage)</code></p>

<p><code>song_preview = Song.new(fromLocalStorage)</code></p>

<p>Then say I have a Song class, where I always want to assign the <code>@time_stamp</code>, but then depending on whether there is <code>cloud_storage</code> or not, assign <code>@cloud_store_spec</code></p>

<pre><code>def initialize(cloud_storage = nil, time_stamp = nil, local_storage = nil)
    @time_stamp = time_stamp || (Time.now.to_f * 1000).to_i.to_s
    @cloud_store_spec = cloud_storage
end 
</code></pre>

<p>I'm thinking of using <code>nil</code> as I have done, however would the code know which is <code>cloud_storage</code> and which is <code>local_storage</co

In [ ]:
i = randint(0, posts.shape[0])
print(f"Topic: {posts.iloc[i]['topic_name']}")
print('Title:', (posts.iloc[i]['Title']))
print('Body:', (posts.iloc[i]['Body']))

Topic: Network and IP Address Handling
Title: Master and multiple child database tables vs. master and a 'properties' table
Body: <p>In my Rails app, I want to allow the user to connect to multiple remote locations via different protocols (FTP, SFTP, S3, Dropbox). Each location has different properties:</p>

<pre><code>FTP: host, port, username, password, is_passive
SFTP: host, port, username, password
S3: bucket, key (not sure these are right; I'm guessing)
Dropbox: username, password (again, I'm guessing)
</code></pre>

<p>What would be the more advantageous database table design for storing this information?</p>

<p><strong>Option 1</strong> (master table with tables for each protocol)</p>

<pre><code>connection
    id
    name
    protocol

ftp_connection
    connection_id
    host
    username
    password
    port
    is_passive

sftp_connection
    connection_id
    host
    username
    password
    port

s3_connection
    connection_id
    bucket
    key

dropbox_connection
  

In [ ]:
i = randint(0, posts.shape[0])
print(f"Topic: {posts.iloc[i]['topic_name']}")
print('Title:', (posts.iloc[i]['Title']))
print('Body:', (posts.iloc[i]['Body']))

Topic: Form Rendering Techniques
Title: Using the gem globalize, how to switch locales for input only and not the whole page?
Body: <p><strong>Context:</strong> For a Ruby on Rails app for bike rentals, I am using the gem globalize to deal with input <code>:description</code> in different languages. </p>

<p><strong>Curent state:</strong> The globalize implementation worked, depending on my locale I am able to store <code>description</code> in a specific language. The input for <code>:description</code> is dealt with on the basis of the locale of an entire webpage.</p>

<p>This means that everything on this page has to change in language in order to store <code>:description</code> in the correct language.</p>

<p>Alternatively, I am also able to show all available locales and show <code>description</code> for each of them. (See also the commented out code below).  </p>

<p><strong>Question:</strong> I'm searching for a way to let the user select a language for <code>:description</code>

In [ ]:
import csv
directory = f"/content/drive/MyDrive/Nikta"
path = os.path.join(directory, 'Data', '50k_sample_posts_data_frame.csv')

posts.to_csv(path, index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)